In [1]:
import torch
import pandas as pd
import numpy as np
#import re
#import nltk
#nltk.download()

from transformers import *

In [2]:
data = pd.read_csv("text_sentences.txt", delimiter='\n', header=None)
data.head()

,0
0,alpha-catenin inhibits beta-catenin signaling ...
1,A binary complex of birch profilin and skeleta...
2,Abnormal immunoreactivity in the tumor tissue ...
3,Absence of alpha-syntrophin leads to structura...
4,"Abundance of actin , talin , alpha 5 and beta ..."


In [3]:
print('Total sentences :',len(data))
max(data[0].str.len())

Total sentences : 1100


646

In [4]:
max_len = 0
j = 0
count=0
for i in range(len(data)):
    if len(data[0][i])>400:
        count=count+1
    if len(data[0][i]) > max_len:
        max_len = len(data[0][i])
        j=i
print('sentences greater a particular length:',count)

sentences greater a particular length: 20


## Below is the tokenization of text for SciBERT

In [5]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

In [6]:
encoded = data[0].apply(lambda x: tokenizer.encode_plus(
    text=x,  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    max_length = 512,  # maximum length of a sentence
    pad_to_max_length=True,  # Add [PAD]s
    truncation = True,
    return_attention_mask = True,  # Generate the attention mask
    #return_tensors = 'pt',  # ask the function to return PyTorch tensors
))

/opt/anaconda3/envs/nlpenv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2016: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [7]:
np.array(encoded[1]['input_ids']).shape

(512,)

In [8]:
#encoded[2]['input_ids']

In [9]:
ids = np.array([encoded[i]['input_ids'] for i in range(len(encoded))])
masked = np.array([encoded[i]['attention_mask'] for i in range(len(encoded))])

In [10]:
input_ids = torch.tensor(ids)  
attention_mask = torch.tensor(masked)

In [11]:
input_ids.shape

torch.Size([1100, 512])

In [12]:
# adata = data.iloc[0:2]
# aencoded = adata[0].apply(lambda x: tokenizer.encode_plus(
#     text=x,  # the sentence to be encoded
#     add_special_tokens=True,  # Add [CLS] and [SEP]
#     max_length = 512,  # maximum length of a sentence
#     pad_to_max_length=True,  # Add [PAD]s
#     truncation = True,
#     return_attention_mask = True,  # Generate the attention mask
#     #return_tensors = 'pt',  # ask the function to return PyTorch tensors
# ))
# aids = np.array([aencoded[i]['input_ids'] for i in range(len(aencoded))])
# amasked = np.array([aencoded[i]['attention_mask'] for i in range(len(aencoded))])
# ainput_ids = torch.tensor(aids)  
# aattention_mask = torch.tensor(amasked)
#aattention_mask.shape

In [13]:
config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
config.output_hidden_states =True

In [14]:
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=config
#                                     output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.

In [15]:
#coment for fine tuning
# model.train()
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [16]:
#running the model to get ourlayers
with torch.no_grad():

    outputs = model(input_ids,attention_mask )

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [24]:
print(len(hidden_states))
print(type(hidden_states))
outputs[0].size()

13
<class 'tuple'>


torch.Size([1100, 512, 768])

In [22]:
#save the above hidden units tensor
#torch.save(hidden_states,'hidden_states_scibert.pt')

In [12]:
# uncomment to load the trained tensors
#stat=torch.load('hidden_states_scibert.pt')
#hidden_states = stat

In [21]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1100
Number of tokens: 512
Number of hidden units: 768


In [25]:
stacked = torch.stack(hidden_states, dim=0)
print(type(stacked))
stacked.size()

<class 'torch.Tensor'>


torch.Size([13, 1100, 512, 768])

In [26]:
#concatinating the last 4 layers
word_embedding = []
for batch in range(len(hidden_states[0])):
    for token in range(len(hidden_states[0][0])):
        cat_vec = torch.cat((stacked[-1][batch][token],
                             stacked[-2][batch][token],
                             stacked[-3][batch][token],
                             stacked[-4][batch][token]),dim=0)
        word_embedding.append(cat_vec)
    

In [27]:
print ('Shape is: %d x %d' % (len(word_embedding), len(word_embedding[0])))

Shape is: 563200 x 3072


In [37]:
outputs[0][1]

tensor([[ 0.0478, -0.4140,  1.0597,  ..., -0.2176, -0.0195,  0.4383],
        [ 0.5467, -0.7389,  0.8160,  ...,  0.1146, -0.1843, -0.0734],
        [ 0.1503, -0.3764, -0.7323,  ..., -0.0040,  0.1554, -0.0617],
        ...,
        [ 0.0815, -0.4239,  0.1776,  ..., -0.3081, -0.6307,  0.4767],
        [ 0.1371, -0.4626,  0.2086,  ..., -0.3079, -0.8089,  0.4637],
        [ 0.1984, -0.0902,  0.3226,  ..., -0.2056, -0.5436,  0.6394]])

In [42]:
hidden_states[12][1]

tensor([[ 0.0478, -0.4140,  1.0597,  ..., -0.2176, -0.0195,  0.4383],
        [ 0.5467, -0.7389,  0.8160,  ...,  0.1146, -0.1843, -0.0734],
        [ 0.1503, -0.3764, -0.7323,  ..., -0.0040,  0.1554, -0.0617],
        ...,
        [ 0.0815, -0.4239,  0.1776,  ..., -0.3081, -0.6307,  0.4767],
        [ 0.1371, -0.4626,  0.2086,  ..., -0.3079, -0.8089,  0.4637],
        [ 0.1984, -0.0902,  0.3226,  ..., -0.2056, -0.5436,  0.6394]])

In [43]:
hidden_states[10][1]

tensor([[-0.9192, -0.2376,  0.9802,  ...,  0.6087,  0.0458,  0.4302],
        [ 0.2473, -0.7353,  0.4240,  ...,  0.8698, -0.3748, -0.0503],
        [ 0.2168, -0.8073, -0.7456,  ...,  0.3421,  0.7219,  0.1095],
        ...,
        [-0.3551, -0.5613,  0.6527,  ..., -0.1454, -0.3965,  0.3426],
        [-0.3456, -0.5752,  0.6691,  ..., -0.1040, -0.6477,  0.3329],
        [-0.5062, -0.2967,  0.5272,  ...,  0.0036, -0.3674,  0.4586]])